In [132]:
INPUT_DATAPATH = "/home/karthikeya/ProjectsSJSU/ML_Project/Data" #path to store files based on search term
SEARCH_TERM = "India"
TWEETS_COUNT = 300

 
consumer_key = 'HImrEBDPQ1v30Ms8yevMAtOc2'
consumer_secret = 'hwmtGELZmLXHAaV1Xok9lkUjNjB3n3fu8SMvnGJv1ofnfMKqd0'
access_token = '354844313-ITXlT4314nPV3WxXjEcgq5xlVecRgK2KItdTjkJV'
access_secret = 'rBvKgdyZaOgcMj5xULpTYZfWaLjlm4wH3qbcUSwJf65tq'


import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [134]:
# Searches user_timeline, selects 10 tweets and puts that into dataframe
#later few parts of data we are interested in is fed into respective arrays
# mean length of tweets is calculated

tweets = api.user_timeline(screen_name=SEARCH_TERM, count=TWEETS_COUNT)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

print(dir(tweets[0]))

# We display the first 10 elements of the dataframe:
display(data.head(10))

data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

display(data.head(10))

mean = np.mean(data['len'])
print("The lenght's average in tweets: {}".format(mean))

Number of tweets extracted: 199.

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


,Tweets
0,Come to vote for the best future coin DGB! htt...
1,"RT @SinghLions: ""Being defeated is often a tem..."
2,RT @officialmcafee: Imagine a world where ever...
3,"@VitalikButerin hi V God, pls follow me back. ..."
4,New VR Streaming Service: BitTorrent Meets Blo...
5,@officialmcafee pls follow me back. Thanks
6,"@officialmcafee John, how do you think about T..."
7,@mashable I like it. Where can we buy?
8,@sweden Congrats to Sweden
9,RT @BillGates: I believe that we can alter the...


,Tweets,len,ID,Date,Source,Likes,RTs
0,Come to vote for the best future coin DGB! htt...,66,979050367701798912,2018-03-28 17:39:32,Twitter for Android,0,0
1,"RT @SinghLions: ""Being defeated is often a tem...",140,975037901128966144,2018-03-17 15:55:25,Twitter for Android,0,5874
2,RT @officialmcafee: Imagine a world where ever...,140,975016427080986625,2018-03-17 14:30:05,Twitter for Android,0,246
3,"@VitalikButerin hi V God, pls follow me back. ...",52,958527589814972416,2018-01-31 02:29:20,Twitter for iPhone,0,0
4,New VR Streaming Service: BitTorrent Meets Blo...,116,955972063872794624,2018-01-24 01:14:35,Twitter for iPhone,2,1
5,@officialmcafee pls follow me back. Thanks,42,955824326720147456,2018-01-23 15:27:32,Twitter for iPhone,0,0
6,"@officialmcafee John, how do you think about T...",80,955824222101692416,2018-01-23 15:27:07,Twitter for iPhone,0,0
7,@mashable I like it. Where can we buy?,38,930590824903016449,2017-11-15 00:18:36,Twitter for iPhone,1,0
8,@sweden Congrats to Sweden,26,930378370264719361,2017-11-14 10:14:23,Twitter for iPhone,1,0
9,RT @BillGates: I believe that we can alter the...,140,930120017475018752,2017-11-13 17:07:47,Twitter for iPhone,0,6541


The lenght's average in tweets: 61.311557788944725


In [137]:
# Before running this code
# make a directory named Data and provide the path of the directory at INPUT_DATAPATH
# set SEARCH_TERM as what you would like to search from twitter "ex: apple"
# which would produce the list of tweets for the term "apple" 
# and also store the twitter data into the file stream_apple.json in the directory of INPUT_DATAPATH

import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import string
import json
import time
import os


class MyListener(StreamListener):
    """Custom StreamListener for streaming data."""

    def __init__(self, data_dir, query):
        query_fname = format_filename(query)
        self.outfile = "%s/stream_%s.json" % (data_dir, query_fname)
        
    def on_data(self, data):
      # removes if a file already exists with same name and creates new file and writes into that
        try:
            try:
                os.remove(self.outfile)
            except OSError:
                pass
            with open(self.outfile, 'a') as f:
                f.write(data)
                print(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            time.sleep(5)

    def on_error(self, status):
        print(status)
        return True


def format_filename(fname):
    """Convert file name into a safe string.
    Arguments:
        fname -- the file name to convert
    Return:
        String -- converted file name
    """
    return ''.join(convert_valid(one_char) for one_char in fname)

'''
def convert_valid(one_char):
    """Convert a character into '_' if invalid.
    Arguments:
        one_char -- the char to convert
    Return:
        Character -- converted char
    """
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status
'''

'\ndef convert_valid(one_char):\n    """Convert a character into \'_\' if invalid.\n    Arguments:\n        one_char -- the char to convert\n    Return:\n        Character -- converted char\n    """\n    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)\n    if one_char in valid_chars:\n        return one_char\n    else:\n        return \'_\'\n\n@classmethod\ndef parse(cls, api, raw):\n    status = cls.first_parse(api, raw)\n    setattr(status, \'json\', json.dumps(raw))\n    return status\n'

In [139]:
twitter_stream = Stream(auth, MyListener(INPUT_DATAPATH, SEARCH_TERM)) #calling stream function to stream in live data
twitter_stream.filter(track=[SEARCH_TERM]) # filters data based on search term on storing

{"created_at":"Mon Apr 16 08:17:45 +0000 2018","id":985794359021813761,"id_str":"985794359021813761","text":"RT @ANI: Tamil Nadu: 16-year-old Vaishnavi from Coimbatore attempted to create five world records in different categories related to variou\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3260941856,"id_str":"3260941856","name":"\ud835\udd86\ud835\udd97\ud835\udd99\ud835\udd8d\ud835\udd9c\ud835\udd86\ud835\udd89\ud835\udd8a\ud835\udd97 \ud83e\udd85","screen_name":"arthvvader","location":"India","url":null,"description":"\u2660\ufe0f\ud83c\udfb5\ud83c\udfb6\ud83d\udc51   \n\nRTs = FYIs   \n\n#Dhoni  \n#CSK","translator_type":"none","protected":false,"verified":false,"followers_count":1171,"friends_count":623,"listed_count":25,"favourites_

{"created_at":"Mon Apr 16 08:17:45 +0000 2018","id":985794360775184384,"id_str":"985794360775184384","text":"RT @Prof_HariOm: #SaveHindusOfJammuFromBJP\nDear Hindus\n3 BJP ministers- Lal, Ganga &amp; Abdul Gani-were sent to Jammu's Hiranagar.\nPM @narendr\u2026","source":"\u003ca href=\"http:\/\/twitter.com\" rel=\"nofollow\"\u003eTwitter Web Client\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":3842947779,"id_str":"3842947779","name":"Venkat68","screen_name":"jayveeyar","location":null,"url":null,"description":"There is no stage to attain in life. To realise that nothing we can do as for us our inner feeling is concerned. These r the two keys for realisation.","translator_type":"none","protected":false,"verified":false,"followers_count":123,"friends_count":636,"listed_count":4,"favourites_count":1334,"statuses_count":24147,"created_at":

{"created_at":"Mon Apr 16 08:17:46 +0000 2018","id":985794363367313408,"id_str":"985794363367313408","text":"RT @mahammed_ansar: How \u201canti-national,\u201d I will be told on social media by people missing the irony. These Hindu \u201cnationalists\u201d who spoke f\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":35746861,"id_str":"35746861","name":"Tavish Alam","screen_name":"mactavish2191","location":"India","url":null,"description":"First Cry on 2nd of Jan \ud83d\ude00.\nYes I am weird, weird is good. Normal is overrated.\nLiberty, Freedom, Plural, Secular.\nThese values I believe in.","translator_type":"none","protected":false,"verified":false,"followers_count":63,"friends_count":379,"listed_count":0,"favourites_count"

{"created_at":"Mon Apr 16 08:17:46 +0000 2018","id":985794364197629952,"id_str":"985794364197629952","text":"RT @BKGoenka: Encouraging to see #India emerging as the destination of choice in Asia by smart #investors.\n\nhttps:\/\/t.co\/DlSpu16dyt","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":118690387,"id_str":"118690387","name":"PRAPHULL CHANDAN JHA","screen_name":"PChandanJha","location":"New Delhi, India","url":"http:\/\/www.epochadvisor.com","description":"#Infra #IndustrialPark #Consulting #Strategy #BusinessPlan #GovernmentRelation #LandAcquisition Plz follow @epochinfra","translator_type":"none","protected":false,"verified":false,"followers_count":367,"friends_count":449,"listed_count":13,"favourites_count":67,"statuses_coun

KeyboardInterrupt: 